## 1. Entrega 01

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

# Pegando o caminho do diretório que estou
rootPath = os.getcwd()
print(rootPath)

# Definindo o diretório com os arquivos metar
metSuperficiePath = rootPath +'\\inputs\\meteoSuperficie'
print(metSuperficiePath)

ezAermetPath = rootPath + '\\inputs\\ezAermet'
print(ezAermetPath) 

#Listando os arquivos dentro do diretório
files = os.listdir(ezAermetPath)
print(files)

C:\Users\anaju\OneDrive\Documentos\GitHub\Controle_AnaPietroski
C:\Users\anaju\OneDrive\Documentos\GitHub\Controle_AnaPietroski\inputs\meteoSuperficie
C:\Users\anaju\OneDrive\Documentos\GitHub\Controle_AnaPietroski\inputs\ezAermet
['meteo.csv']


In [2]:
df = pd.read_csv('C:\\Users\\anaju\\OneDrive\\Documentos\\GitHub\\Controle_AnaPietroski\\inputs\\ezAermet\\METEO.csv')
df

,year,month,day,Julian day,hour,sensible,surface,convective,potential,mixing height,...,anemometer height,temperature,measurement height,precipitation,precipitation amount,relative humidity,station pressure (mb),cloud cover,wind speed.1,cloud cover.1
0,2023,1,1,1,1,-50.0,0.571,-9.0,-9.0,-999,...,14,298.1,2,0,0,83,1010,6,NAD-SFC,NoSubs
1,2023,1,1,1,2,-47.7,0.545,-9.0,-9.0,-999,...,14,298.1,2,0,0,83,1010,6,NAD-SFC,NoSubs
2,2023,1,1,1,3,-54.4,0.622,-9.0,-9.0,-999,...,14,298.1,2,0,0,83,1010,6,NAD-SFC,NoSubs
3,2023,1,1,1,4,-61.1,0.699,-9.0,-9.0,-999,...,14,298.1,2,0,0,83,1010,6,NAD-SFC,NoSubs
4,2023,1,1,1,5,-25.3,0.290,-9.0,-9.0,-999,...,14,298.1,2,0,0,89,1010,6,NAD-SFC,NoSubs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023,12,31,365,20,-43.5,0.496,-9.0,-9.0,-999,...,14,297.1,2,0,0,69,1010,6,NAD-SFC,NoSubs
8756,2023,12,31,365,21,-13.9,0.160,-9.0,-9.0,-999,...,14,297.1,2,0,0,69,1010,6,NAD-SFC,NoSubs
8757,2023,12,31,365,22,-11.4,0.140,-9.0,-9.0,-999,...,14,296.1,2,0,0,78,1010,6,NAD-SFC,NoSubs
8758,2023,12,31,365,23,-999.0,-9.000,-9.0,-9.0,-999,...,-9,296.1,2,9999,-9,999,915,6,NAD,Sub_CC-TT


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Função para determinar a classe de estabilidade atmosférica
def definir_classe_estabilidade(linha):
    cobertura_nuvens = linha['cloud cover ']
    velocidade_vento = linha['wind speed ']
    fluxo_calor_sensivel = linha['sensible']
    
    if fluxo_calor_sensivel > 10:
        insolacao = 'forte'
    elif 3 < fluxo_calor_sensivel <= 10:
        insolacao = 'moderada'
    else:
        insolacao = 'fraca'

    if cobertura_nuvens < 3:
        if velocidade_vento < 2:
            return 'A' if insolacao == 'forte' else 'B'
        elif 2 <= velocidade_vento < 3:
            return 'B' if insolacao == 'forte' else 'C'
        else:
            return 'C'
    elif 3 <= cobertura_nuvens < 7:
        if velocidade_vento < 2:
            return 'B' if insolacao == 'forte' else 'C'
        elif 2 <= velocidade_vento < 5:
            return 'C'
        else:
            return 'D'
    else:
        if velocidade_vento < 2:
            return 'C'
        elif 2 <= velocidade_vento < 5:
            return 'D'
        else:
            return 'E'

In [4]:
#Criando função da estimativa da dispersão lateral e vertical da pluma

def sigmaYZ(x, classe, urbOrRural):
    if urbOrRural == 'urbano':
        if classe in ['A', 'B']:
            sigmaY = 0.32 * x * (1 + 0.0004 * x)**(-0.5)
            sigmaZ = 0.24 * x * (1 + 0.001 * x)**(0.5)
        elif classe == 'C':
            sigmaY = 0.22 * x * (1 + 0.0004 * x)**(-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'D':
            sigmaY = 0.16 * x * (1 + 0.0004 * x)**(-0.5)
            sigmaZ = 0.14 * x * (1 + 0.0003 * x)**(-0.5)
        elif classe in ['E', 'F']:
            sigmaY = 0.11 * x * (1 + 0.0004 * x)**(-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0015 * x)**(-0.5)
        else:
            print('Classe de estabilidade errada')
    elif urbOrRural == 'rural':
        if classe == 'A':
            sigmaY = 0.22 * x * (1 + 0.0001 * x)**(-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'B':
            sigmaY = 0.16 * x * (1 + 0.0001 * x)**(-0.5)
            sigmaZ = 0.12 * x
        elif classe == 'C':
            sigmaY = 0.11 * x * (1 + 0.0001 * x)**(-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0002 * x)**(-0.5)
        elif classe == 'D':
            sigmaY = 0.08 * x * (1 + 0.0001 * x)**(-0.5)
            sigmaZ = 0.06 * x * (1 + 0.0015 * x)**(-0.5)
        elif classe == 'E':
            sigmaY = 0.06 * x * (1 + 0.0001 * x)**(-0.5)
            sigmaZ = 0.03 * x * (1 + 0.0003 * x)**(-1)
        elif classe == 'F':
            sigmaY = 0.04 * x * (1 + 0.0001 * x)**(-0.5)
            sigmaZ = 0.016 * x * (1 + 0.0003 * x)**(-1)
        else:
            print('Classe de estabilidade errada')
            sigmaY, sigmaZ = np.nan, np.nan
    else:
        print('Tipo urbano/rural não reconhecido')
        sigmaY, sigmaZ = np.nan, np.nan
    return sigmaY, sigmaZ

In [5]:
# Função para estimar o deltaH com base na equação de Davidson-Bryant

def DavidsonBryant(d, vs, u, Ts, Tamb):
    deltaH_DavidsonBryant = (d*(vs/u)**(1.4))*(1+(Ts-Tamb)/Ts)
    return deltaH_DavidsonBryant

In [6]:
# Função para estimar o deltaH com base na equação de Holland

def Holland(vs, d, u, p, Ts, Tamb):
    deltaH_Holland = (vs*d/u)*(1.5+(2.68*10**-3)*(p*((Ts-Tamb)/Ts)*d))
    return deltaH_Holland

In [ ]:
def sobrelevacaoBriggs(Vs, d, u, deltaT, Ts, H):
    if deltaT > 0:
        deltaH_Briggs = 1.6 * (Vs * d / u)**0.6 * (deltaT / Ts)**0.4
    else:
        deltaH_Briggs = 0.0
    return deltaH_Briggs


## 2. Entrega 02